In [56]:
import pandas as pd
import numpy as np

In [57]:
#https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t+1'] = df['t'].shift(-1)
print(df)

   t  t+1
0  0  1.0
1  1  2.0
2  2  3.0
3  3  4.0
4  4  5.0
5  5  6.0
6  6  7.0
7  7  8.0
8  8  9.0
9  9  NaN


In [58]:
from pandas import DataFrame
from pandas import concat
#https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(f'var{j+1}(t-{i})') for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(f'var{j+1}(t)') for j in range(n_vars)]
		else:
			names += [(f'var{j+1}(t+{i})') for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# values = [x for x in range(6)]
# data = series_to_supervised(values)
# print(data)

In [59]:
df=pd.read_csv('data/S50-Daily-Y09-22.csv',index_col='Date/Time',parse_dates=['Date/Time'],  \
               dayfirst=True)
df
print(df.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 625 entries, 2020-04-01 to 2022-11-07
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ticker          625 non-null    object 
 1   EMA10           625 non-null    float64
 2   EMA20           625 non-null    float64
 3   MACD10-20       625 non-null    float64
 4   SIGNAL10-20-15  625 non-null    float64
 5   RSI-MA15-5      625 non-null    float64
 6   Volume          625 non-null    float64
 7   Price           625 non-null    float64
dtypes: float64(7), object(1)
memory usage: 43.9+ KB
None


In [61]:
#def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
dfUniv=df[['RSI-MA15-5']]
data = series_to_supervised(dfUniv,1,1)
print(data.head())

            var1(t-1)  var1(t)
Date/Time                     
2020-04-02      37.39    38.94
2020-04-03      38.94    40.29
2020-04-07      40.29    43.61
2020-04-08      43.61    45.70
2020-04-09      45.70    48.25


In [62]:
dfMultiv=df[['MACD10-20','RSI-MA15-5']]
print(dfMultiv.head())

            MACD10-20  RSI-MA15-5
Date/Time                        
2020-04-01     -49.55       37.39
2020-04-02     -41.72       38.94
2020-04-03     -35.14       40.29
2020-04-07     -23.08       43.61
2020-04-08     -14.55       45.70


In [64]:
values = dfMultiv.values
data = series_to_supervised(values,3)

In [65]:
data

,var1(t-3),var2(t-3),var1(t-2),var2(t-2),var1(t-1),var2(t-1),var1(t),var2(t)
3,-49.55,37.39,-41.72,38.94,-35.14,40.29,-23.08,43.61
4,-41.72,38.94,-35.14,40.29,-23.08,43.61,-14.55,45.70
5,-35.14,40.29,-23.08,43.61,-14.55,45.70,-7.57,48.25
6,-23.08,43.61,-14.55,45.70,-7.57,48.25,-0.75,50.34
7,-14.55,45.70,-7.57,48.25,-0.75,50.34,5.06,52.62
...,...,...,...,...,...,...,...,...
620,-2.12,49.68,-0.68,50.79,0.63,52.22,3.06,54.16
621,-0.68,50.79,0.63,52.22,3.06,54.16,4.74,56.37
622,0.63,52.22,3.06,54.16,4.74,56.37,5.95,58.14
623,3.06,54.16,4.74,56.37,5.95,58.14,6.81,59.65
